In [7]:
import nltk
import spacy
from dask.distributed import Client
from textblob import TextBlob

import datetime
import time
import os
from pathlib import Path

In [8]:
pronouns=['I','you','he','she','it','we','they','me','him','her','us','them','what','who','whom','mine','yours','his','hers','ours','theirs','this','that','these','those','who','whom','which','what','whose','whoever','whatever','whichever','whomever','myself','yourself','himself','herself','itself','ourselves','themselves','each other','one another','anything','everybody','another','each','few','many','none','some','all','any','anybody','anyone','everyone','everything','no one','nobody','nothing','none','other','others','several','somebody','someone','something','most','enough','little','more','both','either','neither','one','much','such']

ignored_words = ['part','and','a','the end','end','parts','use','s']

ignored_contexts = ['in the _',
                    'of the _', 
                    '_ of the',
                    'at that _',
                    'at the _',
                    'on the _']

In [19]:
start = time.time()

filename = '/home/saulo/projects/all-pairs/data_small/AA/wiki_00'

#spacy settings
nlp = spacy.load('en_core_web_sm', disable=['ner','textcat'])
nlp.max_length = 1500000
    
#load data
with open(filename, 'r') as myfile:
    data=myfile.read().replace('\n', ' ')
    
#get noun phrases
spacy_obj = nlp(data)
    
del nlp
    
nouns = [chunk.text.lower() for chunk in spacy_obj.noun_chunks]
set_nouns = set([n for n in nouns if n not in pronouns + ignored_words])  #remove pronouns
    
del nouns


all_grams = []

for sent in spacy_obj.sents:
    #get candidate contexts
    textblob_obj = TextBlob(sent.text)
    everygrams = nltk.everygrams(textblob_obj.words,
                                 min_len=3,
                                 max_len=6)
    
    for gram in everygrams:
        all_grams.append(gram)
        
del spacy_obj    
del textblob_obj
    
#build pairs
pairs = []
for gram in all_grams:
    explode_gram = nltk.everygrams(gram,
                                   min_len=1,
                                   max_len=len(gram)-2)  #avoid context patterns with a single word
    
    ctx_candidates = [' '.join(g).lower() for g in explode_gram]
    intersection = list(set(ctx_candidates).intersection(set_nouns))
        
    joined = ' '.join(gram).lower()
    
    for noun in intersection:
        if joined.startswith(noun+" ") or joined.endswith(" "+noun):
            pairs.append((noun, joined.replace(noun,'_',1)))
    
del set_nouns
    
end = time.time()

print(len(pairs))

print(end - start)

54846
11.944045066833496


In [15]:
pairs

[('access', 'title better _'),
 ('access', 'curid=60043944 title better _'),
 ('better access', 'curid=60043944 title _'),
 ('access', 'en.wikipedia.org/wiki curid=60043944 title better _'),
 ('better access', 'en.wikipedia.org/wiki curid=60043944 title _'),
 ('access', 'https en.wikipedia.org/wiki curid=60043944 title better _'),
 ('better access', 'https en.wikipedia.org/wiki curid=60043944 title _'),
 ('access', '_ scheme the'),
 ('access', 'the better _'),
 ('access', '_ scheme also'),
 ('access', 'the better _'),
 ('access', '_ to psychiatrists'),
 ('psychiatrists', 'access to _'),
 ('psychologists', 'to psychiatrists _'),
 ('psychiatrists', '_ psychologists and'),
 ('psychologists', '_ and general'),
 ('health', 'subsidised mental _'),
 ('health', '_ care to'),
 ('better access', '_ scheme the'),
 ('access', '_ scheme the better'),
 ('access', 'scheme the better _'),
 ('better access', 'scheme the _'),
 ('better access', '_ scheme also'),
 ('access', '_ scheme also known'),
 ('ac

In [1]:
import spacy
import time
from textblob import TextBlob
nlp = spacy.load('en_core_web_sm')

filename = '/home/saulo/projects/all-pairs/samples/queen.txt'

with open(filename, 'r') as myfile:
    data=myfile.read().replace('\n', ' ')

In [2]:
doc = nlp(data)
for token in doc:
    print (doc.text)

Queen are a British rock band formed in London in 1970. Their classic line-up was Freddie Mercury (lead vocals and piano), Brian May (lead guitar and vocals), Roger Taylor (drums and vocals), and John Deacon (bass guitar). Their earliest works were influenced by progressive rock, hard rock and heavy metal, but the band gradually ventured into more conventional and radio-friendly works by incorporating further styles, such as arena rock and pop rock.  Before forming Queen, May and Taylor had played together in the band Smile. Mercury was a fan of Smile and encouraged them to experiment with more elaborate stage and recording techniques. He joined in 1970 and suggested the name "Queen". Deacon was recruited before the band recorded their eponymous debut album in 1973. Queen first charted in the UK with their second album, Queen II, in 1974. Sheer Heart Attack later that year and A Night at the Opera in 1975 brought them international success. The latter featured "Bohemian Rhapsody", whic

Queen are a British rock band formed in London in 1970. Their classic line-up was Freddie Mercury (lead vocals and piano), Brian May (lead guitar and vocals), Roger Taylor (drums and vocals), and John Deacon (bass guitar). Their earliest works were influenced by progressive rock, hard rock and heavy metal, but the band gradually ventured into more conventional and radio-friendly works by incorporating further styles, such as arena rock and pop rock.  Before forming Queen, May and Taylor had played together in the band Smile. Mercury was a fan of Smile and encouraged them to experiment with more elaborate stage and recording techniques. He joined in 1970 and suggested the name "Queen". Deacon was recruited before the band recorded their eponymous debut album in 1973. Queen first charted in the UK with their second album, Queen II, in 1974. Sheer Heart Attack later that year and A Night at the Opera in 1975 brought them international success. The latter featured "Bohemian Rhapsody", whic

In [3]:
start = time.time()

blob = TextBlob(data)
nouns = list(set(blob.noun_phrases))

end = time.time()

print(nouns)
print(end - start)

['will rock', 'lead guitar', 'may', "queen 's record sales range", 'ivor novello', 'grammy lifetime', 'arena rock', 'collection', 'british music award', 'songwriters', 'composers', 'heavy metal', 'outstanding contribution', 'international success', 'outstanding song', 'bass guitar', 'paul rodgers', 'british academy', 'mercury', 'rock history', 'progressive rock', 'music video', 'taylor', 'eponymous debut album', 'freddie mercury', 'british rock band', 'adam lambert', 'england', 'john deacon', 'achievement award', 'august', 'smile', 'bites', 'various publications', 'bohemian rhapsody', 'rock', 'aids', 'authors', 'uk', 'queen name', 'brian may', 'deacon', 'knebworth', 'early 1980s', 'greatest hits', 'london', 'music artists', 'live', 'classic line-up', 'compilation album', 'phonographic', 'roll', 'champions', 'band ’ s', 'stadium rock bands', 'roger taylor', 'a night', 'sheer heart attack', 'fame', 'lead vocals', 'dust', "world 's", 'album news', 'hard rock', 'ii', 'times platinum']
7.57

In [2]:
start = time.time()

doc = nlp(data)
nouns = list(set([chunk.text for chunk in doc.noun_chunks]))

end = time.time()

print(nouns)
print(end - start)

['Each member', 'the Opera', 'bass guitar', 'nine weeks', 'their eponymous debut album', 'the best-selling album', 'hit singles', 'Sheer Heart Attack', 'Smile', 'Authors', 'various publications', 'Queen', 'guitar', 'the music video', 'Paul Rodgers', 'number', '300 million records', 'sporting events', 'their second album', 'pop rock', 'the World', 'vocals', 'Their classic line-up', 'Knebworth', 'Estimates', 'Another One Bites', 'heavy metal', 'a fan', 'You', 'international success', 'Outstanding Song Collection', 'progressive rock', '"Bohemian Rhapsody', 'the Queen name', 'the Dust', 'piano', 'the 1985 Live Aid concert', 'further styles', 'British Music Award', 'Greatest Hits', 'recording techniques', 'a complication', 'eight times platinum', 'May', 'the biggest stadium rock bands', 'the US', 'Composers', 'he', 'Freddie Mercury', 'more conventional and radio-friendly works', 'London', 'his last performance', 'Fame', 'rock history', 'they', 'them', 'the band', 'the Outstanding Contributi